In [43]:
#!pip install datasets torch tensorboard


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datasets import load_dataset
import numpy as np

In [45]:
#Chargement du dataset
from datasets import load_dataset

dataset = load_dataset("sander-wood/irishman")

train_data = dataset["train"]
val_data = dataset["validation"]

print("Nombre de chansons (train) :", len(train_data))
print("Nombre de chansons (validation) :", len(val_data))



Nombre de chansons (train) : 214122
Nombre de chansons (validation) : 2162


In [46]:
print(train_data[0]["abc notation"])


X:1
L:1/8
M:4/4
K:Emin
|: E2 EF E2 EF | DEFG AFDF | E2 EF E2 B2 |1 efe^d e2 e2 :|2 efe^d e3 B |: e2 ef g2 fe | 
 defg afdf |1 e2 ef g2 fe | efe^d e3 B :|2 g2 bg f2 af | efe^d e2 e2 ||


**Prétraitement des données**

In [47]:
#Extraction des caractères uniques
train_songs = [item["abc notation"] for item in train_data]

all_text = "".join(train_songs)
vocab = sorted(set(all_text))

print("Nombre de caractères uniques :", len(vocab))

Nombre de caractères uniques : 95


In [48]:
#Mapping caractères ↔ indices
char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = {i: c for c, i in char2idx.items()}

In [49]:
#Vectorisation des chaînes
def vectorize_string(s, char2idx):
    return [char2idx[c] for c in s]

test_vec = vectorize_string(train_songs[0], char2idx)
print(test_vec[:20])

[56, 26, 17, 0, 44, 26, 17, 15, 24, 0, 45, 26, 20, 15, 20, 0, 43, 26, 37, 77]


In [50]:
#Padding des séquences
max_len = max(len(song) for song in train_songs)
print("Longueur maximale :", max_len)

def pad_sequence(s, max_len, pad_char=" "):
    if len(s) < max_len:
        return s + pad_char * (max_len - len(s))
    else:
        return s[:max_len]

Longueur maximale : 2968


**Création du Dataset PyTorch**

In [51]:
#Préparation finale des données
def prepare_data(songs, char2idx, max_len):
    sequences = []
    for song in songs:
        padded = pad_sequence(song, max_len)
        vectorized = vectorize_string(padded, char2idx)
        sequences.append(vectorized)
    return sequences

train_sequences = prepare_data(train_songs, char2idx, max_len)
val_sequences = prepare_data(
    [item["abc notation"] for item in val_data],
    char2idx,
    max_len
)


In [52]:
from torch.utils.data import Dataset, DataLoader
import torch

class MusicDataset(Dataset):
    def __init__(self, sequences, pad_idx, max_len):
        self.pad_idx = pad_idx
        self.max_len = max_len
        self.data = [self.pad_or_truncate(seq) for seq in sequences]
        self.data = torch.tensor(self.data, dtype=torch.long)

    def pad_or_truncate(self, seq):
        if len(seq) > self.max_len:
            return seq[:self.max_len]
        else:
            return seq + [self.pad_idx] * (self.max_len - len(seq))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        x = seq[:-1]   # entrée
        y = seq[1:]    # cible décalée
        return x, y
BATCH_SIZE = 8
MAX_LEN = 300
PAD_IDX = char2idx[" "]   # très important

train_dataset = MusicDataset(train_sequences, PAD_IDX, MAX_LEN)
val_dataset   = MusicDataset(val_sequences, PAD_IDX, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Vérification
x, y = next(iter(train_loader))
print(x.shape, y.shape)  # (8, MAX_LEN-1)


torch.Size([8, 299]) torch.Size([8, 299])


**Implémentation du modèle LSTM**

In [53]:
import torch.nn as nn

class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=pad_idx
        )

        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


In [54]:
# Hyperparamètres demandés (adaptés pour 1 epoch)
EMBEDDING_DIM = 256
HIDDEN_SIZE = 512      # plus léger que 1024
LEARNING_RATE = 5e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MusicRNN(
    vocab_size=len(char2idx),
    embedding_dim=EMBEDDING_DIM,
    hidden_size=HIDDEN_SIZE,
    pad_idx=PAD_IDX
)

model.to(device)


MusicRNN(
  (embedding): Embedding(95, 256, padding_idx=1)
  (lstm): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=95, bias=True)
)

In [55]:
def train_one_epoch(model, train_loader, val_loader, lr, pad_idx):
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    writer = SummaryWriter()

    # ===== TRAIN =====
    model.train()
    train_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)

        loss = criterion(
            outputs.view(-1, outputs.size(-1)),
            y.view(-1)
        )

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # ===== VALIDATION =====
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                y.view(-1)
            )
            val_loss += loss.item()

    val_loss /= len(val_loader)

    writer.add_scalars("Loss", {"Train": train_loss, "Val": val_loss}, 0)
    writer.close()

    torch.save(model.state_dict(), "model_1_epoch.pth")

    print(f"Train Loss : {train_loss:.4f} | Val Loss : {val_loss:.4f}")


**Génération de musique**

In [56]:
train_one_epoch(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    lr=LEARNING_RATE,
    pad_idx=PAD_IDX
)


Train Loss : 1.2787 | Val Loss : 1.2816


In [59]:
import torch
import torch.nn.functional as F
from music21 import converter, midi

# =====================
# Fonction de génération
# =====================
def generate_music(model, start_seq, char2idx, idx2char, length=200, pad_idx=None):
    model.eval()
    device = next(model.parameters()).device

    input_seq = torch.tensor(
        [char2idx.get(c, pad_idx) for c in start_seq],
        dtype=torch.long
    ).unsqueeze(0).to(device)

    generated = start_seq

    with torch.no_grad():
        for _ in range(length):
            output = model(input_seq)
            probs = F.softmax(output[0, -1], dim=0)
            next_idx = torch.multinomial(probs, 1).item()
            next_char = idx2char[next_idx]

            generated += next_char
            input_seq = torch.cat(
                [input_seq, torch.tensor([[next_idx]], device=device)],
                dim=1
            )

    return generated

# =====================
# Générer la musique
# =====================
start_seq = (
    "X:1\n"
    "T:Generated Tune\n"
    "M:4/4\n"
    "K:C\n"
)

generated_music = generate_music(
    model=model,
    start_seq=start_seq,
    char2idx=char2idx,
    idx2char=idx2char,
    length=200,
    pad_idx=PAD_IDX
)

print("=== Musique générée ===")
print(generated_music)

# =====================
# Sauvegarder en fichier .abc
# =====================
with open("generated_music.abc", "w") as f:
    f.write(generated_music)
print("Fichier ABC créé : generated_music.abc")

# =====================
# Convertir en MIDI et jouer
# =====================
abc_stream = converter.parse("generated_music.abc")
mf = midi.translate.streamToMidiFile(abc_stream)
mf.open("generated_music.mid", "wb")
mf.write()
mf.close()
print("Fichier MIDI créé : generated_music.mid")

# Jouer directement (si possible sur votre machine)
abc_stream.show('midi')


=== Musique générée ===
X:1
T:Generated Tune
M:4/4
K:C
|:"C""^A""A"Bb/ag/"^/""C"m"B/"_="g<d"C".F/.E/).G<c"G""G"F/"G""G"Bg"D""Am"{B}ed/"A"A/B/c/d/"Bb"|"Em"{cB}"Ab"Bef"B7""D"Ag"D"A/B/c/d/"E""^C"B,"Bb"Bb"Bb"G"B<b"E"E"B"B<E"Em"B/"E"B7"Be"B"Bg"D""A"Ece"E"e/f/"
Fichier ABC créé : generated_music.abc
Fichier MIDI créé : generated_music.mid


abcFormat: WARNING: Could not get pitch information from note:  m, assuming C
abcFormat: WARNING: Could not get pitch information from note:  _=, assuming C


In [58]:
# BONUS : Augmentation des données musicales
# Étape 1 : Définir les notes musicales ABC

ABC_NOTES = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

# Étape 2 : Fonction de transposition

def transpose_abc(song, shift=1):
    result = ""
    for c in song:
        if c in ABC_NOTES:
            idx = ABC_NOTES.index(c)
            result += ABC_NOTES[(idx + shift) % len(ABC_NOTES)]
        else:
            result += c
    return result

# Étape 3 : Augmenter le dataset par transposition

def augment_by_transposition(songs):
    augmented = []
    for song in songs:
        augmented.append(song)                      # chanson originale
        augmented.append(transpose_abc(song, 1))   # transposition vers le haut
        augmented.append(transpose_abc(song, -1))  # transposition vers le bas
    return augmented

# Étape 4 : Utilisation
train_songs_augmented = augment_by_transposition(train_songs)

print("Avant augmentation :", len(train_songs))
print("Après augmentation :", len(train_songs_augmented))


Avant augmentation : 214122
Après augmentation : 642366
